
## Harmonic Oscillator Wavefunction & Basis Set
<br>
Solving the Schrödinger equation with a potential defined as the quantum Harmonic Oscillator yields a set of wavefunctions that can be used as basis set for quantum mechanical calculations. The Schrödinger eqation with the Haromic Oscillator is defined as follows: 
<br><br>
$$\begin{align}
    \hat{H}\Psi &= E\Psi \\
    \hat{H} &= \frac{-\hbar^2}{2m}\frac{\partial^2}{\partial x^2} + \frac{1}{2}kx^2
\end{align}$$
<br>
The wavefunctions produced are:
<br><br>
$$\begin{align}
    \psi_0 &= c_0 \cdot e^{-\frac{\alpha x^2}{2}} \\
    \psi_1 &= c_0 \cdot \sqrt{2\alpha}r \cdot e^{-\frac{\alpha x^2}{2}} \\ 
    \psi_2 &= c_0 \cdot \frac{2y^2 - 1}{\sqrt{2}} \cdot e^{-\frac{\alpha x^2}{2}} \\
    \psi_3 &= c_0 \cdot \frac{2y^3 - 3y}{\sqrt{3}} \cdot e^{-\frac{\alpha x^2}{2}} \\
    \\
    c_0 &= \left[\frac{\alpha}{\pi}\right]^\frac{1}{4} \\
    y &= \sqrt{\alpha}x \\
    \alpha &= \frac{\mu\omega}{\hbar} \\
    \mu &= \hbox{Reduced Atomic Mass} \\
    \omega &= \frac{\hbox{Diatomic Molecule Vibrational Frequency}}{2\pi}
\end{align}$$
The changing term between $c_0$ and the $e$ terms is a polynomial series known as the Hermite Polynomials, and for each $\psi_n$, there is a unique hermite polynomial to match.
For each of the following $n$ values, the associated Hermite Polynomial, $H_n$, is presented.
$$\begin{matrix}
    n & H_n \\
    0 & 1 \\
    1 & 2y \\
    2 & 4y^2-2 \\
    3 & 8y^3 - 12y 
\end{matrix}$$
<br>
It should also be noted that the basis set must be orthonormal, meaning the functions satisfy the following constraint:
$$\int_{-\infty}^{\infty}{\psi_i \psi_j} = \delta_{i,j}$$
<br>
All information in this notebook originates from the following <a href="http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/hosc5.html">website</a>, and the <i>Franck-Condon Calculations</i> Matematica notebook provided by Dr. Jerry LaRue of Chapman University.    

In [1]:
#Import all Required Classes
from mpmath import exp
from mpmath import mp
from mpmath import mpf
from mpmath import pi
from mpmath import factorial
from mpmath import sqrt
from mpmath import quad as integrate
from mpmath import inf
from mpmath import hermite
from tqdm import tqdm
from IPython.display import display, Math, HTML
import numpy as np
from scipy import constants as consts
from scipy.integrate import quad as integrate
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#Set numerical precision
mp.prec = 150
mp.dps = 70

#Declare All Global Variables Here
figure = {
    "data":[],
    "layout":
        {
           "xaxis":{"title":"Angstroms"},
           "yaxis":{"title":"Wavenumbers"},
           "title":{"text":"Harmonic Oscillator Wavefunctions"}
        },    
}

#Hbar in units of kg m^2 / s
hbar = mpf(consts.hbar)
#Re in units of Angstroms
Re = mpf(1.128323)
#w in units of 1/cm
w = mpf(2169.81358)
#u in units of AMU
u = mpf(12*16/(12+16))

## Note on Units
<br>
Due to numerical issues that are experianced when $e$ is raised to either very small or very large numbers, the unnits in this calculation for $x$ are modified from angstroms to a unitless quantity.
This unitless quantity is then multiplied by a constant in order to achive the final desired units
of wavenumbers from the function. 
<br><br>
In order to allow for these conversions, $\hbar$ units of $\frac{\mbox{kg}\cdot\mbox{m}^2}{\mbox{s}^2}$ are used. Because of the units of $\hbar$, $u$ is converted from AMU to kg, and $\omega$ is converted from wavenumbers to hertz.

In [2]:
#All conversion factors acquired from Google

#Concvert u from AMU to kg
u *= mpf(1.6605) * (mpf(10) ** mpf(-27)) 

print("u in kg: " + str(u))
print()

#convert w from 1/cm to 1/s 
#convert w from 1/cm to 1/m
w *= mpf(100)
#convert from 1/m to 1/s using the speed of light, m/s
#    (1/m) * (m/s)
#    1/s
w *= mpf(consts.c) * 2 * np.pi
print(w)

unitConverter = (u*w) / hbar

print()
print(unitConverter)

u in kg: 1.138628571428571425050435550474500230012109067233815708240025812297835e-26

408717275073503.3739366262372542051390621957571347245077603843599334255

44129490943502555397127.58762326445554812401098418873045962173673169885


In [3]:
#Define all functions here
#########################################################################
    
def rToX(r):
    return (r - Re) / (mpf(10) ** mpf(10))

#########################################################################

#Represents the e^x term in the harmonic oscillator
def eTerm(r):
    x = rToX(r)
    return exp(-unitConverter * (x ** mpf(2)) / 2)

#########################################################################

def hermiteInput(r):
    x = rToX(r)
    return sqrt(unitConverter) * x

#########################################################################

#Function that create a new Harmonic Oscillator Wavefunction
def newHOW(n):
    
    normalization = 1 / sqrt( (2 ** n) * factorial(n) )
    toAU = ( unitConverter * (mpf(10) ** mpf(-20)) / pi ) ** mpf(0.25)
    
    constant = normalization * toAU 
    print("Wave Function: " + str(n) + ", Constant Value " + str(constant))
    print("1/2^n * n!: " + str(normalization))
    print("uw / pi hb ^ 0.25: " + str(toAU))
    print("*"*20)
    print()
    return lambda r : normalization * toAU * eTerm(r) * hermite(n, hermiteInput(r))

In [4]:
basisSize = 3
basis = []

#Build the basis set of the specified size
for n in range(basisSize):
    basis.append( newHOW(n) )

x = np.arange(0, 5, .01)

for index, b in enumerate(basis):
    
    y = []
    for r in x:
        y.append( float(b(r)) )
    
    figure["data"].append(
        {
            "x":x,
            #Convert to wavenumbers
            "y":y,#[yv * 2.2 * pow(10,4) for yv in y],
            "name":"HOW " + str(index)
        }
)

#Compute and display orthonormality of the basis set
overlap = np.zeros([basisSize, basisSize])

for row in tqdm(range(basisSize)):
    for col, b2 in enumerate(basis):
        integrand = lambda r : basis[row](r) * b2(r)
        overlap[row, col] += round(
            integrate(integrand, 0, np.inf, epsabs=pow(10,-5), epsrel=pow(10, -5), limit=pow(10,5))[0],
            7
        )

#Display the Overlap Table
display(Math("\mbox{Overlap Integral:} \int_{-\infty}^{\infty}{\psi_i \psi_j}<br>"))
display(Math( "\qquad\quad\;\;" + "\qquad".join([ "\psi_" + str(col) for col in range(len(basis))]) ))
for row in range(basisSize):
    display(Math("\psi_" + str(row) + "\qquad" + "\qquad".join([str(ovl) for ovl in overlap[row]]) ))

iplot(figure)

Wave Function: 0, Constant Value 3.442664975862117638869367054451982613959774097156269551289930432872994
1/2^n * n!: 1.0
uw / pi hb ^ 0.25: 3.442664975862117638869367054451982613959774097156269551289930432872994
********************

Wave Function: 1, Constant Value 2.434331749785525332052809274498955788900485834625575235554799179114206
1/2^n * n!: 0.7071067811865475244008443621048490392848359376884740365883398689953662
uw / pi hb ^ 0.25: 3.442664975862117638869367054451982613959774097156269551289930432872994
********************

Wave Function: 2, Constant Value 1.217165874892762666026404637249477894450242917312787617777399589557103
1/2^n * n!: 0.3535533905932737622004221810524245196424179688442370182941699344976831
uw / pi hb ^ 0.25: 3.442664975862117638869367054451982613959774097156269551289930432872994
********************



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.34s/it]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
#Harmonic Oscilator Basis Set Class Test
from BasisSets import HOW

how = HOW(1.128323, 2169.81358, 12*16/(12+16), 3)

x, y = how.graphData(0, 5)

figure = {
    
    "data":[],
    "layout":
        {
           "xaxis":{"title":"Angstroms"},
           "yaxis":{"title":"Wavenumbers"},
           "title":{"text":"Harmonic Oscillator Wavefunctions"}
        },    
    
}

for index, yData in enumerate(y):
    figure["data"].append(
        {
            "x":x,
            "y":yData,
            "name":"HOW " + str(index)
        }
    )

iplot(figure)

Graphing Data


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 538.10it/s]
